# A5-Prejudice Remover

In [1]:
## Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv2
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns
from time import time

### Data processing

In [2]:
## Load raw data
raw_data = pd.read_csv('../data/compas-scores-two-years.csv')

In [3]:
## Data cleaning & data processing

# Keep only the "African-American" and "Caucasian" rows
processed_data = raw_data.loc[raw_data['race'].isin(["African-American", "Caucasian"])]
# Remove columns with multiple missing data
processed_data = processed_data[['sex', 'age', 'age_cat', 'race', 'decile_score', 'juv_fel_count', 'juv_misd_count', 'juv_other_count',
            'priors_count', 'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_charge_degree', 'is_recid', 
             'score_text', 'two_year_recid']]
# If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, 
# we can assume that because of data quality reasons, that we do not have the right offense.

processed_data = processed_data.loc[processed_data['days_b_screening_arrest'] <= 30]
processed_data = processed_data.loc[processed_data['days_b_screening_arrest'] >= -30]
# The recidivist flag (is_recid) should be -1 if we could not find a compas case at all.

processed_data = processed_data.loc[processed_data['is_recid'] != -1]
# Ordinary traffic offenses (c_charge_degree = 'O') will not result in Jail time and hence are removed 
# (only two of them).

processed_data = processed_data.loc[processed_data['c_charge_degree'] != 'O']
# score_text shouldn't be 'N/A'

processed_data = processed_data.loc[processed_data['score_text'] != 'N/A']

processed_data['length_of_stay'] = (pd.to_datetime(processed_data['c_jail_out'])-pd.to_datetime(processed_data['c_jail_in'])).apply(lambda x: x.days)

processed_data = processed_data.drop(columns=['c_jail_in', 'c_jail_out'])

In [4]:
processed_data

,sex,age,age_cat,race,decile_score,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_charge_degree,is_recid,score_text,two_year_recid,length_of_stay
1,Male,34,25 - 45,African-American,3,0,0,0,0,-1.0,F,1,Low,1,10
2,Male,24,Less than 25,African-American,4,0,0,1,4,-1.0,F,1,Low,1,1
6,Male,41,25 - 45,Caucasian,6,0,0,0,14,-1.0,F,1,Medium,1,6
8,Female,39,25 - 45,Caucasian,1,0,0,0,0,-1.0,M,0,Low,0,2
10,Male,27,25 - 45,Caucasian,4,0,0,0,0,-1.0,F,0,Low,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7207,Male,30,25 - 45,African-American,2,0,0,0,0,-1.0,M,1,Low,1,0
7208,Male,20,Less than 25,African-American,9,0,0,0,0,-1.0,F,0,High,0,0
7209,Male,23,Less than 25,African-American,7,0,0,0,0,-1.0,F,0,Medium,0,1
7210,Male,23,Less than 25,African-American,3,0,0,0,0,-1.0,F,0,Low,0,1


In [5]:
# Replace the values of the sensitive attribute race as follows: Caucasian -> 1, African-American -> 0
processed_data = processed_data.replace({'race': 'Caucasian'}, 1)
processed_data = processed_data.replace({'race': 'African-American'}, 0)

# Replace the values of sex as follows
processed_data = processed_data.replace({'sex': 'Male'}, 1)
processed_data = processed_data.replace({'sex': 'Female'}, 0)

# Replace the values of age_cat as follows
processed_data = processed_data.replace({'age_cat': 'Less than 25'}, 0)
processed_data = processed_data.replace({'age_cat': '25 - 45'}, 1)
processed_data = processed_data.replace({'age_cat': 'Greater than 45'}, 2)

# Replace the values of c_charge_degree as follows
processed_data = processed_data.replace({'c_charge_degree': 'F'}, 0)
processed_data = processed_data.replace({'c_charge_degree': 'M'}, 1)

# Replace the values of score_text as follows
processed_data = processed_data.replace({'score_text': 'Low'}, 0)
processed_data = processed_data.replace({'score_text': 'Medium'}, 1)
processed_data = processed_data.replace({'score_text': 'High'}, 2)

In [6]:
# Check whether there are NaN values in the final dataset as well as the number of unique values per column

unique_NAN_df = pd.DataFrame(columns=['column name', '# of unique values', '# of NaN values'])
for item in processed_data.columns:
    unique_NAN_df = unique_NAN_df.append({
        'column name': item, 
        '# of unique values': len(processed_data[item].unique()),
        '# of NaN values': sum(processed_data[item].isna() == True)}, ignore_index = True)
    
unique_NAN_df = unique_NAN_df.style.hide_index()
unique_NAN_df

column name,# of unique values,# of NaN values
sex,2,0
age,62,0
age_cat,3,0
race,2,0
decile_score,10,0
juv_fel_count,9,0
juv_misd_count,10,0
juv_other_count,8,0
priors_count,36,0
days_b_screening_arrest,56,0


In [7]:
# move two_year_recid to the end

cols = list(processed_data.columns.values)
cols.pop(cols.index('two_year_recid'))
processed_data = processed_data[cols+['two_year_recid']]

In [8]:
# move race to the first column

race_column = processed_data.pop('race')
processed_data.insert(0, 'race', race_column)

In [9]:
processed_data = processed_data.drop(columns=['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count'])

In [10]:
processed_data

,race,sex,age_cat,decile_score,priors_count,days_b_screening_arrest,c_charge_degree,is_recid,score_text,length_of_stay,two_year_recid
1,0,1,1,3,0,-1.0,0,1,0,10,1
2,0,1,0,4,4,-1.0,0,1,0,1,1
6,1,1,1,6,14,-1.0,0,1,1,6,1
8,1,0,1,1,0,-1.0,1,0,0,2,0
10,1,1,1,4,0,-1.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
7207,0,1,1,2,0,-1.0,1,1,0,0,1
7208,0,1,0,9,0,-1.0,0,0,2,0,0
7209,0,1,0,7,0,-1.0,0,0,1,1,0
7210,0,1,0,3,0,-1.0,0,0,0,1,0


In [11]:
# save final data set to csv
# processed_data.to_csv("../output/processed-compas-scores-two-years.csv", index=False)

### Split the data

In [12]:
data = np.array(processed_data)
y = np.array(data[:,-1]).flatten()
data = data[:,:-1]
sensitive = data[:,0]
data = preprocessing.scale(data)
data = data[:,1:]

In [13]:
sensitive_idx = np.array(np.where(sensitive==1))[0].flatten()
nonsensitive_idx = np.array(np.where(sensitive!=1))[0].flatten()
data_sensitive = data[sensitive_idx,:]
data_nonsensitive = data[nonsensitive_idx,:]
y_sensitive = y[sensitive_idx]
y_nonsensitive = y[nonsensitive_idx]

In [14]:
# split sensitive data into training, validation, and testing sets

X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(data_sensitive, y_sensitive, test_size= 0.2, random_state=42)
X_train_s, X_valid_s, y_train_s, y_valid_s = train_test_split(X_train_s, y_train_s, test_size = 0.25, random_state=42)

In [15]:
# split non-sensitive data into training, validation, and testing sets

X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(data_nonsensitive, y_nonsensitive, test_size= 0.2, random_state=42)
X_train_n, X_valid_n, y_train_n, y_valid_n = train_test_split(X_train_n, y_train_n, test_size = 0.25, random_state=42)

In [16]:
# create final training, validation, and testing sets

X_train = np.concatenate((X_train_s, X_train_n))
X_valid = np.concatenate((X_valid_s, X_valid_n))
X_test = np.concatenate((X_test_s, X_test_n))

Y_train = np.concatenate((y_train_s, y_train_n))
Y_valid = np.concatenate((y_valid_s, y_valid_n))
Y_test = np.concatenate((y_test_s, y_test_n))

### Prejudice Remover 

In [17]:
# pip install aif360

In [18]:
#pip install tabulate
from tabulate import tabulate

In [19]:
import sys

sys.path.append('../lib/')
import A5_Prejudice_remover

sys.path.append('../lib/')
from EvalMetrics import *

sys.path.append('../lib/')
%run '../lib/A5_Prejudice_remover.py'

sys.path.append('../lib/')
%run '../lib/EvalMetrics.py'

In [20]:
privileged_race = np.array([0]) # African-American
privileged_sex = np.array([1]) # Male

In [21]:
from aif360.datasets import StandardDataset

data_orig = StandardDataset(processed_data, 
                              label_name='two_year_recid', 
                              favorable_classes=[1],
                              protected_attribute_names=['race', 'sex'],
                              privileged_classes=[privileged_race,privileged_sex]
                              )

In [22]:
np.random.seed(42)

data_train, data_test = data_orig.split([0.8], shuffle=True) # 5:1
data_train, data_valid = data_train.split([0.75], shuffle=True) # 5:1 

In [23]:
debiased_model = PrejudiceRemover(sensitive_attr="race", eta = 25.0)
debiased_model.fit(data_train) 

/Applications/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Applications/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

#### Accuracy and calibration of PR on Training and Validations Sets

In [61]:
# get predictions for the training dataset
pred_test = debiased_model.predict(data_test).features
np.shape(pred_test)
test_race = pd.DataFrame(pred_test).iloc[:,0]
test_race = pd.DataFrame(test_race)
test_race = test_race.rename(columns={0: 'race'})

test_true_y = pd.DataFrame(data_test.labels.ravel()).iloc[:,0] #true y
test_true_y = pd.DataFrame(test_true_y)
test_true_y = test_true_y.rename(columns={0: 'y_true'})

test_pred = pd.DataFrame(debiased_model.predict(data_test).scores>= 0.5) # predicted y
test_pred = test_pred.rename(columns={0: 'y_pred'})

df = test_race.join(test_pred,how="left")
df = df.join(test_true_y,how="left")
df['y_pred']=df['y_pred'].astype(float)
df

,race,y_pred,y_true
0,0.0,1.0,1.0
1,0.0,1.0,1.0
2,0.0,1.0,1.0
3,0.0,0.0,0.0
4,0.0,1.0,1.0
...,...,...,...
1051,0.0,1.0,1.0
1052,1.0,0.0,0.0
1053,0.0,1.0,1.0
1054,0.0,1.0,1.0


In [62]:
pred_PR_test_s = df['y_pred'][df['race']==1]

pred_PR_test_n = df['y_pred'][df['race']==0]

pred_PR_test = np.concatenate((pred_PR_test_s, pred_PR_test_n))

y_PR_test_s = df['y_true'][df['race']==1]

y_PR_test_n = df['y_true'][df['race']==0]

y_test = np.concatenate((y_test_s, y_test_n))

In [55]:
np.shape(y_PR_test_s)


(429,)

In [56]:
np.shape(pred_PR_test_s)


(429,)

In [60]:
pred_PR_test_s

6       0
7       0
8       1
12      1
14      1
       ..
1041    1
1043    0
1044    1
1049    0
1052    0
Name: y_pred, Length: 429, dtype: int64

In [53]:
np.shape(pred_PR_test_n)

(627,)

In [54]:
np.shape(y_PR_test_n)

(627,)

In [66]:
np.array(pred_PR_test_s)

array([0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
       1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 0., 0.

In [67]:
get_model_performance(np.array(pred_PR_test_s),np.array(y_test_s))

ValueError: Found input variables with inconsistent numbers of samples: [429, 421]

In [46]:
plot_model_performance(pred_PR_test_s, pred_PR_test_n, pred_PR_test, y_test_s, y_test_n, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [421, 429]